In [1]:
import polars

polars.enable_string_cache()

In [2]:
from autofeat import source

dataset = source.from_kaggle(
    name="santander-customer-transaction-prediction",
)


In [3]:
from autofeat.transform import Cast, Encode, Require, Select

transform = (
    Require(lambda table: table.name == "train.csv")
    .then(Select(exclude=["target"]))
    .then(Cast())
    .then(Encode())
)

In [4]:
filters = (
    dataset.table("train.csv")
    .column("ID_code")
    .sample
)

target = (
    dataset.table("train.csv")
    .column("target")
    .sample
)

In [5]:
features = dataset.features(
    filters=filters,
    transform=transform,
)

In [6]:
df = features.collect()

In [7]:
import sklearn.feature_selection
import xgboost

selector = sklearn.feature_selection.SelectFromModel(
    xgboost.XGBRegressor(missing=float("inf")),
    max_features=10,
)

selector.fit(
    X=df.to_numpy(),
    y=target.to_numpy(),
)

selection = [
    df.columns[i]
    for i, selected in enumerate(selector.get_support())
    if selected
]

selection

['var_0 from train.csv']